# Common logic

In [59]:
# DEFINES
project_name = 'lazyut'


In [60]:
%run Modules/some_helpers.ipynb

class ProjectInfo:
    def __init__(self):
        self.list_info = []
        self.n_commits = 10
        self.n_iter = 3
        self.test_mode = TestMode()
        self.compiler_c = GCC_C
        self.compiler_cxx = GCC_CXX
        self.cmake_generator = 'Ninja'
        self.when_prepare = 'before configure'
        self.lazyut_extra_options = ''
        self.first_commit = True
        self.initialize_lazyut_fun = lambda pro: None
        
    @staticmethod
    def dir_lazyut_base():
        return os.path.join(os.path.expanduser("~"),'src','builds','lazyut_files')
    @staticmethod
    def dir_build_base():
        return os.path.join(os.path.expanduser("~"),'src','builds')
    
    def check_attributes(self):
        attributes = ['name', 'branch', 'directory', 
                      'src_rel_dir', 'test_rel_dir', 
                      'prepare_fun', 
                      'configure_fun', 'build_fun', 'test_fun']
        for a in attributes:
            if not hasattr(self, a):
                raise RuntimeWarning('{}: no attr {}'.format(self.name, a))
    
    def new_iteration(self):
        self.iteration_info = IterationInfo()
        self.iteration_info.setCommit(self.current_commit)
        self.iteration_info.stage = 'idle'
        self.list_info.append(self.iteration_info) # add object by reference
    def setTestMode(self, v):
        self.test_mode.value = v
        self.iteration_info.clear()
    def new_files_dir(self):
        return os.path.join(self.dir_lazyut(), 'files')
    def last_files_dir(self):
        return os.path.join(self.dir_lazyut(), 'last_version', 'files')
    def src_dir(self):
        return os.path.join(self.directory, self.src_rel_dir)
    def test_dir(self):
        return os.path.join(self.directory, self.test_rel_dir)
    def prepare(self):
        self.iteration_info.stage = 'prepare'
        prepare_sdt = timeit.default_timer()
        if self.test_mode.isLazy():
            self.prepare_fun(self)
        self.iteration_info.prepare_ts = timeit.default_timer() - prepare_sdt
    def configure(self):
        self.iteration_info.stage = 'configure'
        configure_sdt = timeit.default_timer()
        self.configure_fun(self)
        self.iteration_info.configure_ts = timeit.default_timer() - configure_sdt
    def build(self):
        self.iteration_info.stage = 'build'
        build_sdt = timeit.default_timer()
        self.build_fun(self)
        self.iteration_info.build_ts = timeit.default_timer() - build_sdt
    def test(self):
        self.iteration_info.stage = 'test'
        test_sdt = timeit.default_timer()
        self.test_fun(self)
        self.iteration_info.test_ts = timeit.default_timer() - test_sdt
    def initialize_lazyut(self):
        self.initialize_lazyut_fun(self)
        execute_lazyut(self)
    def dir_lazyut(self):
        return os.path.join(ProjectInfo.dir_lazyut_base(), self.name)
    def dir_build(self):
        return os.path.join(ProjectInfo.dir_build_base(), self.name)
    def clear_lazyut_files(self):
        print('remove files {}'.format(self.dir_lazyut()))
        shutil.rmtree(self.dir_lazyut(), ignore_errors=True)
    def clean_up(self):
        # delete untracked files
        # -d for directories, -f for force, 
        # -x for don't use standart .gitignore rules (remove build products)
        execute('git clean -dfx', cwd = self.directory)
        # clear local changes for project
        execute('git reset --hard', cwd = self.directory)

class IterationInfo:
    __st_idle = 0
    __st_finished_failed = 1
    __st_finished_success = 2
    # global info
    total_ts_retest_all = 0
    total_count_retest_all = 0
    total_failed_all = 0
    total_ts_retest_lazyut = 0
    total_count_retest_lazyut = 0
    total_failed_lazyut = 0
    @staticmethod
    def addFailed(mode):
        if mode == TestMode.retest_all:
            IterationInfo.total_failed_all += 1
        elif mode == TestMode.test_affected:
            IterationInfo.total_failed_lazyut += 1
        else:
            print('addFailed::unknown')
    @staticmethod
    def avg_retest_all():
        if IterationInfo.total_count_retest_all:
            return IterationInfo.total_ts_retest_all / IterationInfo.total_count_retest_all
        return -1.0
    @staticmethod
    def avg_retest_lazyut():
        if IterationInfo.total_count_retest_lazyut:
            return IterationInfo.total_ts_retest_lazyut / IterationInfo.total_count_retest_lazyut
        return -1.0
    
    # instance info
    def __init__(self):
        self.state = IterationInfo.__st_idle
    def clear(self):
        self.prepare_ts = 0
        self.configure_ts = 0
        self.build_ts = 0
        self.test_ts = 0
    def setCommit(self, commit):
        self.commit = commit
    def setDurationAll(self, ts):
        self.retest_all_ts = ts
        IterationInfo.total_ts_retest_all += ts
        IterationInfo.total_count_retest_all += 1
    def setDurationLazyut(self, ts):
        self.retest_lazyut_ts = ts
        IterationInfo.total_ts_retest_lazyut += ts
        IterationInfo.total_count_retest_lazyut += 1
    def setDurationLazyutAnalysis(self, ts):
        self.lazyut_analysis = ts
    def setDuration(self, ts, mode):
        if mode == TestMode.retest_all:
            self.setDurationAll(ts)
        elif mode == TestMode.test_affected:
            self.setDurationLazyut(ts)
        else:
            print('setDuration::unknown')
    def success(self):
        self.state = IterationInfo.__st_finished_success
    def fail(self):
        self.state = IterationInfo.__st_finished_failed
    
class TestMode:
    unknown = 0
    retest_all = 1
    test_affected = 2
    
    @staticmethod
    def common_directory():
        return 'default'
    
    def __init__(self):
        self.value = TestMode.unknown
    def name(self):
        if self.value == TestMode.unknown:
            return ''
        if self.value == TestMode.retest_all:
            return 're-test all'
        if self.value == TestMode.test_affected:
            return 'lazy testings'
    def directory(self):
        if self.value == TestMode.unknown:
            return ''
        if self.value == TestMode.retest_all:
            return 'default'
        if self.value == TestMode.test_affected:
            return project_name
    def isLazy(self):
        return self.value == TestMode.test_affected
    
def clear_out_file(project):
    with open(project.name + '_output.txt', 'w') as f: 
        f.write('COMMIT,MODE,CONFIGURE,ANALYSIS,BUILD,TEST\n')
    
def append_out_file(project, line):
    with open(project.name + '_output.txt', 'a+') as f: 
        f.write(line)
        f.write('\n')
        
def is_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) == 0

def moveFilesLazyUT(project):
    # get list of files
    files = os.listdir(project.new_files_dir())
    # create last_version/files/ directory
    os.makedirs(project.last_files_dir(), exist_ok=True)
    # move files to last_version/files/ directory and overwrite if exists
    for f in files:
        shutil.move(os.path.join(project.new_files_dir(), f), os.path.join(project.last_files_dir(), f))
    
def execute_lazyut(project):
    cmd_str = lazyut_cmd + ' -v -r {} -s {} -t {} -o {} -i {} {}'.format(
                project.directory,
                project.src_rel_dir, project.test_rel_dir, 
                project.new_files_dir(), project.last_files_dir(),
                project.lazyut_extra_options)
    print('lazyut_cmd ' + cmd_str)
#     exit(1)
    execute(cmd_str)
    
def run_lazyut(project):
    analysis_sdt = timeit.default_timer()
    execute_lazyut(project)
    analysis_edt = timeit.default_timer()
    project.iteration_info.setDurationLazyutAnalysis(analysis_edt - analysis_sdt)

def build_project(project):
    # create build directory if not created
    if hasattr(project, 'build_dir'):
        os.makedirs(project.build_dir, exist_ok=True)
    # setup project
    try:
        #  clean project from any changes
        project.clean_up()
        
        if project.when_prepare == 'before configure':
            project.prepare()
            
        # configure (run cmake, etc.)
        project.configure()
        
        if project.when_prepare == 'after configure':
            project.prepare()
        
        # build project
        project.build()
    except Exception as e:
        project.iteration_info.fail()
        message = 'Stage {} failed! {}'.format(project.iteration_info.stage, 
                                               project.iteration_info.commit) + '\nwhat: ' + str(e)
        print(message)
        raise e
        
def test_project(project):
    test_sdt = timeit.default_timer()
    try:
        project.test()
    except Exception as e:
        project.iteration_info.fail()
        message = 'Tests failed! ' + project.iteration_info.commit + '\nwhat: ' + str(e)
        print(message)
        raise e
        
    test_edt = timeit.default_timer()
    test_duration = test_edt - test_sdt
    project.iteration_info.test_ts = test_duration
    output = '{},{},{:.5},{:.5},{:.5},{:.5}'.format(
        project.iteration_info.commit,
        project.test_mode.name(), 
        project.iteration_info.configure_ts,
        project.iteration_info.lazyut_analysis,
        project.iteration_info.build_ts, project.iteration_info.test_ts)
    print(output)
    append_out_file(project, output)
    project.iteration_info.setDuration(test_duration, project.test_mode.value)


    
def run_iteration(project):
    # build default, then build LazyUT in the same build directory
    project.setTestMode(TestMode.test_affected)
    build_project(project)
    test_project(project)
    project.setTestMode(TestMode.retest_all)
    build_project(project)
    test_project(project)

        
def run_on_commit(project):
    try:
        # run re-test all and re-test affected (LazyUT) several times
        # and store evaluation times
        for i in range(0, project.n_iter):
            print('Iteration {}/{}:'.format(i+1, project.n_iter))
            project.new_iteration()
            run_iteration(project)
        # print average testing times
        output = 're-test all {:.5}[sec]'.format(IterationInfo.avg_retest_all())
        output += ' re-test affected (LazyUT) {:.5}[sec]'.format(IterationInfo.avg_retest_lazyut())
        print(output)
    except Exception as e:
        print('COMMIT {} SKIPPED'.format(project.current_commit))
        exit(1) # TODO remove
    
def git_checkout(project):
    project.clean_up()
    execute('git checkout {}'.format(project.current_commit), cwd = project.directory)
    
# -- SCRIPT --
def apply_lazyut(project):
    # clears output.txt
    clear_out_file(project)
    # clears lazyut/ directory
    project.clear_lazyut_files()
    # get the list of commits
    rev_list_str = check_output('git rev-list -n {} {}'.format(
                        project.n_commits, project.branch), cwd = project.directory, shell = True)
    rev_list = rev_list_str.splitlines()[::-1]
    print('{}: total commits: {}'.format(project.name, len(rev_list)))
    
    # use previous commit to initialize LazyUT
    project.current_commit = rev_list.pop(0).decode('utf-8')
    git_checkout(project)
    project.initialize_lazyut()
    moveFilesLazyUT(project)
    # iterate over commits, starting with 'the first commit'
    for commit_bytes in rev_list:
        commit = commit_bytes.decode('utf-8')
        project.current_commit = commit
        print('{}: evaluate commit {}'.format(project.name, project.current_commit))
        # checkout to commit
        git_checkout(project)
        # evaluate multiple iteration on commit
        run_on_commit(project)
        # move LazyUT files from lazyut/files/ to lazyut/last_version/files/
        moveFilesLazyUT(project)
    print('{}: FINISHED!'.format(project.name))

### CMAKE

In [61]:
def CMAKE_GLOB_SRCS(line, tmpf):
    # Add only affected test files
    if 'file(GLOB srcs' in line:
        tmpf.write('    file(STRINGS lazyut/files/tests_affected.txt srcs)\n')
        return
    if 'file(GLOB hdrs' in line:
        tmpf.write('    set(hdrs "") # empty list\n')
        return
    # Just copy line
    tmpf.write(line)

def prepareCmakeProject(project, method):
    # evaluate list of tests
    run_lazyut(project)
    # install list of tests
    cmakelists_test = os.path.join(project.test_dir(), 'CMakeLists.txt')
    # We are going to modify file, so will use temporary file
    tmpf = tempfile.NamedTemporaryFile(mode='r+')
    # Open for read
    with open(cmakelists_test, 'r') as cmakelists_txt:
        # Firstly check if it is not prepared yet
        # And return if it is
        for line in cmakelists_txt:
            if project_name in line:
                return
        cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
        # Create temporary file read/write
        # Copy input file to temporary file, modifying as we go
        for line in cmakelists_txt:
            method(line, tmpf)
    tmpf.seek(0) # Rewind temporary file to beginning
    # Reopen for write
    with open(cmakelists_test, 'w') as cmakelists_txt:
        for line in tmpf:
            cmakelists_txt.write(line)
    
def configure_cmake(project, extra_flags = ""):
    execute('cmake -G "{}" -DCMAKE_CXX_COMPILER={} -DBUILD_TESTS=ON {} {}'.format(
        project.cmake_generator, 
        project.compiler_cxx, 
        extra_flags,
        project.directory), 
            cwd = project.build_dir)
def build_cmake(project):
    execute('cmake --build .', timeout=None, cwd = project.build_dir)

In [62]:
lazyut_cmd = find_existent('C:/Projects/Study/build-LazyRT-5_9-Default/bin/lazyut.exe', 
                           'D:/Study/build-LazyUT-5_7-Debug/bin/lazyut.exe',
                           'C:/experiments/build-LazyUT-gcc5-Default/bin/lazyut.exe',
                           '/home/astyco/src/builds/lazyut/bin/lazyut')
GCC_C =  find_existent('C:/Qt/Tools/mingw530_32/bin/gcc.exe', 
                       'D:/Soft/Qt/Tools/mingw530_32/bin/gcc.exe',
                       'C:/Soft/mingw_gcc_8_1/mingw64/bin/gcc.exe',
                       '/usr/bin/gcc')
GCC_CXX = find_existent('C:/Qt/Tools/mingw530_32/bin/g++.exe', 
                        'D:/Soft/Qt/Tools/mingw530_32/bin/g++.exe',
                        'C:/Soft/mingw_gcc_8_1/mingw64/bin/g++.exe',
                        '/usr/bin/g++')

### CppCheck

In [63]:
project_cppcheck = ProjectInfo()
def run_on_cppcheck():
    def prepareCppCheck(project):
        cmakelists_test = os.path.join(project.test_dir(), 'CMakeLists.txt')
        # We are going to modify file, so will use temporary file
        tmpf = tempfile.NamedTemporaryFile(mode='r+')
        # Open for read
        with open(cmakelists_test, 'r') as cmakelists_txt:
            # Firstly check if it is not prepared yet
            # And return if it is
            for line in cmakelists_txt:
                if 'shlwapi' in line:
                    tmpf.close()
                    return 
            cmakelists_txt.seek(0) # Rewind cmakelists file to beginning
            # Create temporary file read/write
            # Copy input file to temporary file, modifying as we go
            for line in cmakelists_txt:
                tmpf.write(line)
                # Link missed libraries (MINGW)
                if 'add_executable(testrunner' in line:
                    tmpf.write('    if (MINGW)\n')
                    tmpf.write('        target_link_libraries(testrunner shlwapi)\n')
                    tmpf.write('    endif()\n')
        tmpf.seek(0) # Rewind temporary file to beginning
        # Reopen for write
        with open(cmakelists_test, 'w') as cmakelists_txt:
            for line in tmpf:
                cmakelists_txt.write(line)
        tmpf.close() 
    def prepare_cppcheck(project):
        if project.test_mode.isLazy():
            prepareCmakeProject(project, CMAKE_GLOB_SRCS)
            affected_tests_txt = os.path.join(project.new_files_dir(), 'tests_affected.txt')
            if is_zero_file(affected_tests_txt):
                raise RuntimeWarning('is_zero_file(affected_tests_txt)')
        prepareCppCheck(project)

    def configure_cppcheck(project):
        configure_cmake(project)

    def build_cppcheck(project):
        build_cmake(project)

    def test_cppcheck(project):
        testrunner = os.path.join('bin', 'testrunner')
        execute(testrunner, cwd = project.build_dir())

    project_cppcheck.name = 'CppCheck'
    project_cppcheck.same_build_dir_flag = True
    # project_cppcheck.cmake_generator = 'Unix Makefiles'
    project_cppcheck.branch = 'master'
    project_cppcheck.n_commits = 2
    project_cppcheck.n_iter = 2
    project_cppcheck.directory = find_existent('C:/Projects/cppcheck', 
                                               'D:/Study/TestCoveredProjects/cppcheck',
                                               '/home/astyco/src/cppcheck',
                                               'C:/experiments/cppcheck')
    print('cppcheck project directory: {}'.format(project_cppcheck.directory))

    project_cppcheck.lazyut_extra_options = '--ignore="cfg/,cfg\\,synthetic/,synthetic\\\\"'
    project_cppcheck.src_rel_dir = 'lib'
    project_cppcheck.test_rel_dir = 'test'

    project_cppcheck.builds_dir = project_cppcheck.directory + '/builds'

    project_cppcheck.prepare_fun = prepare_cppcheck
    project_cppcheck.configure_fun = configure_cppcheck
    project_cppcheck.build_fun = build_cppcheck
    project_cppcheck.test_fun = test_cppcheck


    project_cppcheck.check_attributes()
    apply_lazyut(project_cppcheck)


### Botan

In [64]:
import re

project_botan = ProjectInfo()
def run_on_botan(project):
    def prepare_botan(project):
        # firstly evaluate affected tests
        run_lazyut(project)
        # then modify cmake scripts
        pro_file_path = os.path.join(project.directory, 'CMakeLists.txt')
        # Create temporary file read/write
        tmpf = tempfile.NamedTemporaryFile(mode='r+')
        # Open Makefile for read
        with open(pro_file_path, 'r') as pro_file:
            # Copy input file to temporary file, modifying as we go
            in_the_botan_test_sources = False
            for line in pro_file:
                if re.search(r'set[\s]*\([\s]*BOTAN_TESTS', line):
                    # begin of the list of test source file
                    in_the_botan_test_sources = True
                if in_the_botan_test_sources:
                    if ')' in line:
                        in_the_botan_test_sources = False
                        tmpf.write('file(STRINGS {}/tests_affected.txt BOTAN_TESTS)\n'.format(project.new_files_dir()))
                else:
                    tmpf.write(line)
        tmpf.seek(0) # Rewind temporary file to beginning
        # Reopen Makefile for write
        with open(pro_file_path, 'w') as pro_file:
            for line in tmpf:
                pro_file.write(line)
        tmpf.close() 

    def configure_botan(project):
        execute('./configure.py --with-cmake && cmake -GNinja .', cwd = project.build_dir)

    def build_botan(project):
        execute('cmake --build .', cwd = project.build_dir)

    def test_botan(project):
        execute('./botan-test', cwd = project.build_dir)
        
    def initialize_lazyut_botan(project):
        configure_botan(project)

    project.name = 'botan'
    project.branch = 'master'
    project.when_prepare = 'after configure'
    project.n_commits = 50
    project.n_iter = 1
    project.directory = find_existent('/home/astyco/src/botan') 
    project.build_dir = project.directory

    project.lazyut_extra_options = '--ignore="{}" --include-paths="{}" -d {}'.format(
        'botan/build.h', 'build/include', 
        os.path.join(ProjectInfo.dir_lazyut_base()), 'botan_test_data_deps.txt')
    project.src_rel_dir = os.path.join('build', 'include')
    project.test_rel_dir = os.path.join('src', 'tests')

    project.prepare_fun = prepare_botan
    project.configure_fun = configure_botan
    project.build_fun = build_botan
    project.test_fun = test_botan
    project.initialize_lazyut_fun = initialize_lazyut_botan

    project.check_attributes()
    apply_lazyut(project)

# abseil-cpp

In [65]:
import re
# 14.11.2018 - 10.02.2019 34 commit
project_abseil = ProjectInfo()
def run_on_abseil(project):
    def prepare_abseil(project):
        # firstly evaluate affected tests
        run_lazyut(project)
        # then modify cmake scripts
        script_path = os.path.join(project.directory, 'CMake', 'AbseilHelpers.cmake')
        # Create temporary file read/write
        tmpf = tempfile.NamedTemporaryFile(mode='r+')
        # Open for read
        with open(script_path, 'r') as script_file:
            # Copy input file to temporary file, modifying as we go
            for line in script_file:
                if re.search(r'set(_NAME "absl_${ABSL_CC_TEST_NAME}")', line):
                    # begin of the list of test source file
                    tmpf.write('''
  file (STRINGS .../tests_affected.txt ABSL_TESTS)\n
  set (ABSL_TESTS absl/base/internal/compressed_tuple_test.cc)\n
  set (B_SKIP true)\n
  foreach (SRC ${ABSL_CC_TEST_SRCS})\n
    foreach (AFF_SRC ${ABSL_TESTS})\n
      if (AFF_SRC MATCHES "${SRC}$")\n
        message ("match " ${AFF_SRC} ${SRC}})\n
        set (B_SKIP false)\n
      endif()\n
    endforeach()\n
  endforeach ()\n
  if (${B_SKIP})\n
    return()\n
  endif()\n''')
                    tmpf.write(line)
        tmpf.seek(0) # Rewind temporary file to beginning
        # Reopen Makefile for write
        with open(script_path, 'w') as script_file:
            for line in tmpf:
                script_file.write(line)
        tmpf.close() 

    def configure_abseil(project):
        execute('cmake -DABSL_USE_GOOGLETEST_HEAD=ON -DABSL_RUN_TESTS=ON -GNinja {}'.format(
            project.directory), cwd = project.dir_build())

    def build_abseil(project):
        execute('cmake --build .', cwd = project.dir_build())

    def test_abseil(project):
        execute('./ctest', cwd = project.dir_build())

    project.name = 'abseil-cpp'
    project.branch = 'master'
    project.when_prepare = 'before configure'
    project.n_commits = 30
    project.n_iter = 1
    project.directory = find_existent('/home/astyco/src/{}'.format(project.name)) 

    project.lazyut_extra_options = '--include-paths="{}"'.format(
        'build/include')
    project.src_rel_dir = ""
    project.test_rel_dir = ""

    project.prepare_fun = prepare_abseil
    project.configure_fun = configure_abseil
    project.build_fun = build_abseil
    project.test_fun = test_abseil

    project.check_attributes()
    apply_lazyut(project)

# Script

In [66]:
# run_on_botan(project_botan)
# run_on_cppcheck()
run_on_abseil(project_abseil)

print('script finished %s' % str(datetime.datetime.now()) )

remove files /home/astyco/src/builds/lazyut_files/abseil-cpp
abseil-cpp: total commits: 30
lazyut_cmd /home/astyco/src/builds/lazyut/bin/lazyut -v -r /home/astyco/src/abseil-cpp -s  -t  -o /home/astyco/src/builds/lazyut_files/abseil-cpp/files -i /home/astyco/src/builds/lazyut_files/abseil-cpp/last_version/files --include-paths="build/include"


RuntimeWarning: operation failed /home/astyco/src/builds/lazyut/bin/lazyut -v -r /home/astyco/src/abseil-cpp -s  -t  -o /home/astyco/src/builds/lazyut_files/abseil-cpp/files -i /home/astyco/src/builds/lazyut_files/abseil-cpp/last_version/files --include-paths="build/include" {}

In [ ]:
projects = [project_cppcheck, project_botan]
for proj in projects:
    for iteration_info in proj.list_info:
        print('{}: {} commit'.format(proj.name, iteration_info.commit))
        print('{}: {} retest_all_ts'.format(iteration_info.retest_all_ts))
        print('{}: {} retest_lazyut_ts'.format(iteration_info.retest_lazyut_ts))
        print('{}: {} lazyut_analysis'.format(iteration_info.lazyut_analysis))
        print()
    print('------------')
    